In [1]:
from omegaconf import OmegaConf

In [3]:
conf = OmegaConf.create()
conf

{}

In [7]:
conf = OmegaConf.create({"k" : "v", "list" : [1, {"a": "1", "b": "2", 3: "c"}]})
print(OmegaConf.to_yaml(conf, resolve=True))

k: v
list:
- 1
- a: '1'
  b: '2'
  3: c



In [11]:
conf = OmegaConf.load("ldm/config/kl-f4/config.yaml")
conf

{'model': {'base_learning_rate': 4.5e-06, 'target': 'ldm.models.autoencoder.AutoencoderKL', 'params': {'monitor': 'val/rec_loss', 'embed_dim': 3, 'lossconfig': {'target': 'ldm.modules.losses.LPIPSWithDiscriminator', 'params': {'disc_start': 50001, 'kl_weight': 1e-06, 'disc_weight': 0.5}}, 'encoder': {'double_z': True, 'z_channels': 3, 'resolution': 256, 'ch': 128, 'in_channels': 3, 'out_ch': 3, 'ch_mult': [1, 2, 4], 'num_res_blocks': 2, 'attn_resolutions': [], 'dropout': 0.0}, 'ddconfig': {'double_z': True, 'z_channels': 3, 'resolution': 256, 'ch': 128, 'in_channels': 3, 'out_ch': 3, 'ch_mult': [1, 2, 4], 'num_res_blocks': 2, 'attn_resolutions': [], 'dropout': 0.0}}}, 'data': {'target': 'main.DataModuleFromConfig', 'params': {'batch_size': 10, 'wrap': True, 'train': {'target': 'ldm.data.openimages.FullOpenImagesTrain', 'params': {'size': 384, 'crop_size': 256}}, 'validation': {'target': 'ldm.data.openimages.FullOpenImagesValidation', 'params': {'size': 384, 'crop_size': 256}}}}}

In [13]:
conf.model.target

'ldm.models.autoencoder.AutoencoderKL'

In [14]:
conf.new_values = {"state": "happy"}
conf

{'model': {'base_learning_rate': 4.5e-06, 'target': 'ldm.models.autoencoder.AutoencoderKL', 'params': {'monitor': 'val/rec_loss', 'embed_dim': 3, 'lossconfig': {'target': 'ldm.modules.losses.LPIPSWithDiscriminator', 'params': {'disc_start': 50001, 'kl_weight': 1e-06, 'disc_weight': 0.5}}, 'encoder': {'double_z': True, 'z_channels': 3, 'resolution': 256, 'ch': 128, 'in_channels': 3, 'out_ch': 3, 'ch_mult': [1, 2, 4], 'num_res_blocks': 2, 'attn_resolutions': [], 'dropout': 0.0}, 'ddconfig': {'double_z': True, 'z_channels': 3, 'resolution': 256, 'ch': 128, 'in_channels': 3, 'out_ch': 3, 'ch_mult': [1, 2, 4], 'num_res_blocks': 2, 'attn_resolutions': [], 'dropout': 0.0}}}, 'data': {'target': 'main.DataModuleFromConfig', 'params': {'batch_size': 10, 'wrap': True, 'train': {'target': 'ldm.data.openimages.FullOpenImagesTrain', 'params': {'size': 384, 'crop_size': 256}}, 'validation': {'target': 'ldm.data.openimages.FullOpenImagesValidation', 'params': {'size': 384, 'crop_size': 256}}}}, 'new_v

In [15]:
OmegaConf.merge(conf, OmegaConf.create({"a": {"b": 10}}))

{'model': {'base_learning_rate': 4.5e-06, 'target': 'ldm.models.autoencoder.AutoencoderKL', 'params': {'monitor': 'val/rec_loss', 'embed_dim': 3, 'lossconfig': {'target': 'ldm.modules.losses.LPIPSWithDiscriminator', 'params': {'disc_start': 50001, 'kl_weight': 1e-06, 'disc_weight': 0.5}}, 'encoder': {'double_z': True, 'z_channels': 3, 'resolution': 256, 'ch': 128, 'in_channels': 3, 'out_ch': 3, 'ch_mult': [1, 2, 4], 'num_res_blocks': 2, 'attn_resolutions': [], 'dropout': 0.0}, 'ddconfig': {'double_z': True, 'z_channels': 3, 'resolution': 256, 'ch': 128, 'in_channels': 3, 'out_ch': 3, 'ch_mult': [1, 2, 4], 'num_res_blocks': 2, 'attn_resolutions': [], 'dropout': 0.0}}}, 'data': {'target': 'main.DataModuleFromConfig', 'params': {'batch_size': 10, 'wrap': True, 'train': {'target': 'ldm.data.openimages.FullOpenImagesTrain', 'params': {'size': 384, 'crop_size': 256}}, 'validation': {'target': 'ldm.data.openimages.FullOpenImagesValidation', 'params': {'size': 384, 'crop_size': 256}}}}, 'new_v